In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from numpy import array
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer

In [2]:
#read the data
data = pd.read_excel('../bestsellers with categories.xlsx')

In [3]:
# Data statistics
data.describe()

,User Rating,Reviews,Price,Year
count,550.000000,550.000000,550.000000,550.000000
mean,4.618364,11953.281818,13.100000,2014.000000
std,0.226980,11731.132017,10.842262,3.165156
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,4058.000000,7.000000,2011.000000
50%,4.700000,8580.000000,11.000000,2014.000000
75%,4.800000,17253.250000,16.000000,2017.000000
max,4.900000,87841.000000,105.000000,2019.000000


In [4]:
# The data :)
data

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
...,...,...,...,...,...,...,...
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction


In [5]:
# Computes the average price for each author and replaces the $0 price books with the average price
jk = data[(data['Author'] == 'Jeff Kinney') & (data['Price'] != 0)]
jk_ave_price = jk['Price'].mean()

sm = data[(data['Author'] == 'Stephenie Meyer') & (data['Price'] != 0)]
sm_ave_price = sm['Price'].mean()

hl = data[(data['Author'] == 'Harper Lee') & (data['Price'] != 0)]
hl_ave_price = hl['Price'].mean()

rh = data[(data['Author'] == 'RH Disney') & (data['Price'] != 0)]
rh_ave_price = rh['Price'].mean()

ac = data[(data['Author'] == 'Alice Schertle') & (data['Price'] != 0)]
ac_ave_price = ac['Price'].mean()

consti = data[(data['Name'] == 'The Constitution of the United States') & (data['Price'] != 0)]
consti_ave_price = consti['Price'].mean()

In [6]:
data.loc[(data['Price'] == 0) & (data['Author'] == 'Jeff Kinney'), 'Price'] = jk_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'Stephenie Meyer'), 'Price'] = sm_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'Harper Lee'), 'Price'] = hl_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'RH Disney'), 'Price'] = rh_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'Alice Schertle'), 'Price'] = ac_ave_price
data.loc[(data['Price'] == 0) & (data['Name'] == 'The Constitution of the United States'), 'Price'] = consti_ave_price


print(data[data['Author'] == 'RH Disney'])
print(data[data['Author'] == 'Alice Schertle'])
print(data[data['Name'] == 'The Constitution of the United States'])


#drop NAN values
data = data.dropna()


                            Name     Author  User Rating  Reviews  Price  \
116  Frozen (Little Golden Book)  RH Disney          4.7     3642    NaN   
193         JOURNEY TO THE ICE P  RH Disney          4.6      978    NaN   

     Year    Genre  
116  2014  Fiction  
193  2014  Fiction  
                  Name          Author  User Rating  Reviews  Price  Year  \
219  Little Blue Truck  Alice Schertle          4.9     1884    NaN  2014   

       Genre  
219  Fiction  
                                      Name                             Author  \
358  The Constitution of the United States  Delegates of the ConstitutionalÂ…   

     User Rating  Reviews  Price  Year        Genre  
358          4.8     2774    NaN  2016  Non Fiction  


In [7]:
data[data['Price'] == 0]
data.info()
data = data.reset_index()
data = data.drop("index", axis=1)
data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         546 non-null    object 
 1   Author       546 non-null    object 
 2   User Rating  546 non-null    float64
 3   Reviews      546 non-null    int64  
 4   Price        546 non-null    float64
 5   Year         546 non-null    int64  
 6   Genre        546 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 34.1+ KB


,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,Non Fiction
...,...,...,...,...,...,...,...
541,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,Fiction
542,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8.0,2016,Non Fiction
543,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8.0,2017,Non Fiction
544,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8.0,2018,Non Fiction


In [8]:
counters = {}
dropnames = []
names = data['Name']
for i in range(len(names)):
    if names[i] in counters:
        counters[names[i]] += 1
        dropnames.append(i)
    else:
        counters[names[i]] = 1
data_dropped = data.drop(dropnames)
data_dropped['Occurences'] = counters.values()
data_dropped = data_dropped.reset_index()
data_dropped = data_dropped.drop("index", axis=1)
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,Non Fiction,1
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,Fiction,1
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,Non Fiction,1
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,Fiction,1
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,Non Fiction,1
...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,Fiction,1
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,Non Fiction,1
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,Fiction,5
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,Fiction,1


In [9]:
# Creates a new combined name and author column
data_dropped['Name and Author'] = data_dropped['Name'] + str(' ') + data_dropped['Author']
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences,Name and Author
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,Non Fiction,1,10-Day Green Smoothie Cleanse JJ Smith
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,Fiction,1,11/22/63: A Novel Stephen King
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,Non Fiction,1,12 Rules for Life: An Antidote to Chaos Jordan...
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,Fiction,1,1984 (Signet Classics) George Orwell
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,Non Fiction,1,"5,000 Awesome Facts (About Everything!) (Natio..."
...,...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,Fiction,1,Winter of the World: Book Two of the Century T...
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,Non Fiction,1,Women Food and God: An Unexpected Path to Almo...
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,Fiction,5,Wonder R. J. Palacio
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,Fiction,1,Wrecking Ball (Diary of a Wimpy Kid Book 14) J...


In [10]:
stop_words = nltk.corpus.stopwords.words('english')
stop_words
def clean_text(document):
    tokenizer = RegexpTokenizer(r'\w+')
    pst = PorterStemmer()
    words = tokenizer.tokenize(document)
    lemmatizer = WordNetLemmatizer() # Creates a lemmatizer object
    words = [lemmatizer.lemmatize(w) for w in words]
    words = [pst.stem(w.lower()) for w in words if w.lower() not in stop_words]
    sentence = ' '.join(words)
    return sentence

In [11]:
# Cleans the combined names of the titles and authors
all_words = data_dropped['Name and Author']
clean_names_both = []
for i in range(len(all_words)):
    clean = clean_text(all_words[i])
    clean_names_both.append(clean)
clean_names_both = np.array(clean_names_both)
data_dropped['Clean Name Both'] = clean_names_both
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences,Name and Author,Clean Name Both
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,Non Fiction,1,10-Day Green Smoothie Cleanse JJ Smith,10 day green smoothi cleans jj smith
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,Fiction,1,11/22/63: A Novel Stephen King,11 22 63 novel stephen king
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,Non Fiction,1,12 Rules for Life: An Antidote to Chaos Jordan...,12 rule life antidot chao jordan b peterson
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,Fiction,1,1984 (Signet Classics) George Orwell,1984 signet classic georg orwel
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,Non Fiction,1,"5,000 Awesome Facts (About Everything!) (Natio...",5 000 awesom fact everyth nation geograph kid ...
...,...,...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,Fiction,1,Winter of the World: Book Two of the Century T...,winter world book two centuri trilog ken follett
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,Non Fiction,1,Women Food and God: An Unexpected Path to Almo...,women food god unexpect path almost everyth ge...
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,Fiction,5,Wonder R. J. Palacio,wonder r j palacio
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,Fiction,1,Wrecking Ball (Diary of a Wimpy Kid Book 14) J...,wreck ball diari wimpi kid book 14 jeff kinney


In [12]:
# Encodes the genres to be numerical
genres = data_dropped['Genre']
values = array(genres)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
data_dropped['Genre'] = integer_encoded
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences,Name and Author,Clean Name Both
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,1,1,10-Day Green Smoothie Cleanse JJ Smith,10 day green smoothi cleans jj smith
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,0,1,11/22/63: A Novel Stephen King,11 22 63 novel stephen king
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,1,1,12 Rules for Life: An Antidote to Chaos Jordan...,12 rule life antidot chao jordan b peterson
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,0,1,1984 (Signet Classics) George Orwell,1984 signet classic georg orwel
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,1,1,"5,000 Awesome Facts (About Everything!) (Natio...",5 000 awesom fact everyth nation geograph kid ...
...,...,...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,0,1,Winter of the World: Book Two of the Century T...,winter world book two centuri trilog ken follett
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,1,1,Women Food and God: An Unexpected Path to Almo...,women food god unexpect path almost everyth ge...
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,0,5,Wonder R. J. Palacio,wonder r j palacio
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,0,1,Wrecking Ball (Diary of a Wimpy Kid Book 14) J...,wreck ball diari wimpi kid book 14 jeff kinney


In [13]:
# Creates a new feature that is the product of rating and reviews for the total stars to account for 
# the amount of reviews for each rating
data_dropped['Total Stars'] = data_dropped['User Rating'] * data_dropped['Reviews']
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences,Name and Author,Clean Name Both,Total Stars
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,1,1,10-Day Green Smoothie Cleanse JJ Smith,10 day green smoothi cleans jj smith,81545.0
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,0,1,11/22/63: A Novel Stephen King,11 22 63 novel stephen king,9439.2
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,1,1,12 Rules for Life: An Antidote to Chaos Jordan...,12 rule life antidot chao jordan b peterson,89201.3
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,0,1,1984 (Signet Classics) George Orwell,1984 signet classic georg orwel,100692.8
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,1,1,"5,000 Awesome Facts (About Everything!) (Natio...",5 000 awesom fact everyth nation geograph kid ...,36792.0
...,...,...,...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,0,1,Winter of the World: Book Two of the Century T...,winter world book two centuri trilog ken follett,48420.0
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,1,1,Women Food and God: An Unexpected Path to Almo...,women food god unexpect path almost everyth ge...,5468.4
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,0,5,Wonder R. J. Palacio,wonder r j palacio,103800.0
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,0,1,Wrecking Ball (Diary of a Wimpy Kid Book 14) J...,wreck ball diari wimpi kid book 14 jeff kinney,46123.7


In [14]:
clean_names = data_dropped['Clean Name Both']
clean_names

0                   10 day green smoothi cleans jj smith
1                            11 22 63 novel stephen king
2            12 rule life antidot chao jordan b peterson
3                        1984 signet classic georg orwel
4      5 000 awesom fact everyth nation geograph kid ...
                             ...                        
342     winter world book two centuri trilog ken follett
343    women food god unexpect path almost everyth ge...
344                                   wonder r j palacio
345       wreck ball diari wimpi kid book 14 jeff kinney
346    badass stop doubt great start live awesom life...
Name: Clean Name Both, Length: 347, dtype: object

In [15]:
vec = TfidfVectorizer()
tv_matrix = vec.fit_transform(clean_names)
vocab = vec.get_feature_names()
words_df = pd.DataFrame(np.round(tv_matrix.toarray(), 3), columns=vocab)
words_df

,000,10,100,11,111,12,13,14,140,150,...,write,year,york,young,zelda,zhi,zinczenko,zone,zoo,zusak
0,0.000,0.342,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000,0.000,0.0,0.426,0.0,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.000,0.0,0.000,0.0,0.404,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.297,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
345,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.429,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
frames = [data_dropped, words_df]
final_df = pd.concat(frames, axis = 1)
final_df

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences,Name and Author,Clean Name Both,...,write,year,york,young,zelda,zhi,zinczenko,zone,zoo,zusak
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,1,1,10-Day Green Smoothie Cleanse JJ Smith,10 day green smoothi cleans jj smith,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,0,1,11/22/63: A Novel Stephen King,11 22 63 novel stephen king,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,1,1,12 Rules for Life: An Antidote to Chaos Jordan...,12 rule life antidot chao jordan b peterson,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,0,1,1984 (Signet Classics) George Orwell,1984 signet classic georg orwel,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,1,1,"5,000 Awesome Facts (About Everything!) (Natio...",5 000 awesom fact everyth nation geograph kid ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,0,1,Winter of the World: Book Two of the Century T...,winter world book two centuri trilog ken follett,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,1,1,Women Food and God: An Unexpected Path to Almo...,women food god unexpect path almost everyth ge...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,0,5,Wonder R. J. Palacio,wonder r j palacio,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,0,1,Wrecking Ball (Diary of a Wimpy Kid Book 14) J...,wreck ball diari wimpi kid book 14 jeff kinney,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
data_nonalpha = final_df.drop(['Name', 'Author', 'Name and Author', 'Clean Name Both'], axis=1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_nonalpha)
data_scaled_df = pd.DataFrame(data_scaled)
column_names = ['User Rating', 'Review', 'Price', 'Year', 'Genre', 'Occurences', 'Total Stars']
column_names = column_names + vocab
data_scaled_df.columns = column_names
data_scaled_df 

,User Rating,Review,Price,Year,Genre,Occurences,Total Stars,000,10,100,...,write,year,york,young,zelda,zhi,zinczenko,zone,zoo,zusak
0,0.8750,0.197178,0.067308,0.7,1.0,0.000000,0.193075,0.0,0.914439,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.8125,0.022949,0.201923,0.2,0.0,0.000000,0.021992,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.8750,0.215730,0.134615,0.9,1.0,0.000000,0.211241,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.8750,0.243577,0.048077,0.8,0.0,0.000000,0.238507,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.9375,0.086875,0.105769,1.0,1.0,0.000000,0.086891,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0.7500,0.122124,0.134615,0.3,0.0,0.000000,0.114481,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,0.5625,0.014407,0.096154,0.1,1.0,0.000000,0.012571,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344,0.9375,0.245866,0.076923,0.4,0.0,0.444444,0.245879,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
345,1.0000,0.106783,0.067308,1.0,0.0,0.000000,0.109032,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
data_scaled_df.to_csv('preprocessdffixed.csv')